In [37]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text

In [38]:
instruments = ['hc','rb','i','j','jm','au','ag','v','ru','l','pp','bu','TA','FG','MA',
               'y','p','m','a','c','cs','jd','RM','CF','SR','OI']
start_date = 20180101
table = "AdjustedFuturesDaily"
engine = create_engine("sqlite:///../data/FuturesMarketData.db")
in_binds = ", ".join([f":sym{i}" for i in range(len(instruments))])
sql = text(f"""
        SELECT *, (ClosePrice * factor_multiply) as adjclose
        FROM {table}
        WHERE TradingDay >= {start_date}
            AND Instrument IN ({in_binds})
            AND method = 'OpenInterest'
    """)

params = {"start": start_date} | {f"sym{i}": s for i, s in enumerate(instruments)}

with engine.begin() as conn:
    df = pd.read_sql(sql, conn, params=params)

df = df.pivot(index="TradingDay", columns="Instrument")
df["adjclose"]


Instrument,CF,FG,MA,OI,RM,SR,TA,a,ag,au,...,jd,jm,l,m,p,pp,rb,ru,v,y
TradingDay,,,,,,,,,,,,,,,,,,,,,
20180102,14723.312083,2044.243353,3187.277950,5870.018972,3622.435790,3951.124672,6409.453051,4143.605484,3174.812591,248.445148,...,3682.631235,1337.485979,12780.084953,6227.348248,3739.096470,15452.700709,2716.422467,7786.682521,4861.770518,4510.188152
20180103,14644.996593,2033.057343,3172.012164,5889.771254,3631.935185,3956.484855,6381.865735,4166.378803,3170.744405,249.289897,...,3640.522146,1351.480600,12805.760663,6245.365892,3734.884195,15447.817311,2670.844909,7762.247324,4854.599765,4508.627534
20180104,14693.943774,2056.827614,3198.182082,5927.480155,3614.519628,3924.993781,6476.122398,4177.765463,3160.980758,248.089464,...,3633.822973,1366.474838,12825.017446,6238.609276,3760.157847,15503.162494,2677.856841,7664.506540,4894.038906,4544.521765
20180105,14850.574754,2041.446850,3188.368363,5950.823760,3630.351953,3931.023987,6471.524512,4181.181461,3165.048944,248.845292,...,3625.209750,1359.977335,12805.760663,6249.870303,3764.370123,15477.117702,2663.131784,7650.931431,4897.624283,4544.521765
20180108,14855.469472,2114.155916,3262.516465,5907.727873,3609.769931,3935.714147,6538.193859,4199.400116,3165.862581,249.156515,...,3557.260993,1389.965810,12786.503880,6231.852659,3726.459644,15511.301491,2677.155648,7642.786365,4897.624283,4491.460728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20191225,10235.219679,2450.390868,2075.510711,6421.548746,3854.585317,3716.075319,6669.186159,3314.283424,3308.878432,289.498504,...,3727.844501,1566.202538,10552.153301,6957.642422,3423.864766,15011.191123,3655.007318,4954.526716,5184.511302,4430.568014
20191226,10312.089523,2460.412917,2072.633386,6492.842705,3827.488057,3710.612511,6679.942911,3315.150583,3350.513828,289.784819,...,3717.689654,1553.281197,10595.223314,6942.417821,3448.757372,15150.960128,3701.153752,4950.662031,5196.484769,4456.201629
20191227,10488.890163,2477.116331,2106.202182,6576.018991,3852.891739,3754.314977,6776.753678,3322.087859,3327.046605,290.778502,...,3640.512813,1560.081903,10588.044979,6939.880388,3565.300026,15204.871030,3713.739143,4966.120772,5192.493614,4565.481778


In [39]:
px = df["adjclose"].sort_index()
px = px.where(px > -1)

lookback = 15
skip = 1

# L = 15
# s = 1
# logp = np.log(px)

# mode = "simple"
# signal = logp.shift(s) - logp.shift(L - s)
# signal

# mode = "linear"
log_return = np.log(px / px.shift(1))

weights = np.arange(lookback, 0, -1, dtype=float)
weights /= weights.sum()

signal = (
    log_return
    .rolling(lookback)
    .apply(lambda x: np.dot(x, weights), raw=True)
)
signal

Instrument,CF,FG,MA,OI,RM,SR,TA,a,ag,au,...,jd,jm,l,m,p,pp,rb,ru,v,y
TradingDay,,,,,,,,,,,,,,,,,,,,,
20180102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20180103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20180104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20180105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20180108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20191225,0.003514,0.002706,0.002417,0.000622,0.000448,0.000759,0.002585,0.000346,-0.002681,-0.001578,...,-0.009725,0.001239,0.000779,0.000723,0.004481,-0.001273,0.002995,0.001907,0.002241,0.004527
20191226,0.002794,0.004523,0.000380,0.000990,0.000422,0.000210,0.000614,-0.000675,-0.000080,-0.000949,...,-0.009993,0.000507,0.001233,-0.000017,0.003800,-0.001399,0.003625,-0.001605,0.001737,0.003853
20191227,0.000744,0.004401,0.001990,0.002087,0.000397,0.001026,0.001521,-0.000910,0.001070,0.000102,...,-0.011354,0.000311,0.000317,0.000213,0.003194,-0.002127,0.002668,-0.002634,0.001590,0.003824


In [40]:
window = 15
trade_percent = 0.2
gross_target = 1.0
hold_period = 2
data = df["adjclose"]

low = data.quantile(trade_percent, axis=1)
high = data.quantile(1 - trade_percent, axis=1)

longs = (data.ge(high, axis=0)).astype(float)
shorts = (data.le(low, axis=0)).astype(float) * -1

weights = longs + shorts
weights = weights.sub(weights.mean(axis=1), axis=0)

gross = weights.abs().sum(axis=1)
scale = gross_target / gross.replace(0, np.nan)
position = weights.mul(scale, axis=0).fillna(0)
position

Instrument,CF,FG,MA,OI,RM,SR,TA,a,ag,au,...,jd,jm,l,m,p,pp,rb,ru,v,y
TradingDay,,,,,,,,,,,,,,,,,,,,,
20180102,0.083333,0.000000,0.000000,0.000000,0.0,0.0,0.083333,0.0,0.0,-0.083333,...,0.0,-0.083333,0.083333,0.083333,0.0,0.083333,0.0,0.083333,0.0,0.0
20180103,0.083333,0.000000,0.000000,0.000000,0.0,0.0,0.083333,0.0,0.0,-0.083333,...,0.0,-0.083333,0.083333,0.083333,0.0,0.083333,0.0,0.083333,0.0,0.0
20180104,0.083333,-0.083333,0.000000,0.000000,0.0,0.0,0.083333,0.0,0.0,-0.083333,...,0.0,-0.083333,0.083333,0.083333,0.0,0.083333,0.0,0.083333,0.0,0.0
20180105,0.083333,0.000000,0.000000,0.000000,0.0,0.0,0.083333,0.0,0.0,-0.083333,...,0.0,-0.083333,0.083333,0.083333,0.0,0.083333,0.0,0.083333,0.0,0.0
20180108,0.083333,0.000000,0.000000,0.000000,0.0,0.0,0.083333,0.0,0.0,-0.083333,...,0.0,-0.083333,0.083333,0.083333,0.0,0.083333,0.0,0.083333,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20191225,0.083333,0.000000,-0.083333,0.000000,0.0,0.0,0.083333,0.0,0.0,-0.083333,...,0.0,-0.083333,0.083333,0.083333,0.0,0.083333,0.0,0.000000,0.0,0.0
20191226,0.083333,0.000000,-0.083333,0.000000,0.0,0.0,0.083333,0.0,0.0,-0.083333,...,0.0,-0.083333,0.083333,0.083333,0.0,0.083333,0.0,0.000000,0.0,0.0
20191227,0.083333,0.000000,-0.083333,0.083333,0.0,0.0,0.083333,0.0,0.0,-0.083333,...,0.0,-0.083333,0.083333,0.083333,0.0,0.083333,0.0,0.000000,0.0,0.0


In [41]:
price_diff = data.diff(periods=1)
pos_shift = position.shift(1).fillna(0)
pnl = pos_shift * price_diff

# Portfolio PnL
pnl_ptf = pnl.sum(axis=1)

# Turnover (sum of absolute position changes)
turnover = (position - pos_shift).abs().sum(axis=1)